In [1]:
# File Authorship Information
__author__ = """Matteo Lulli, Luca Biferale, Giacomo Falcucci, 
                Mauro Sbragaglia, Dong YangL and Xiaowen Shan"""
__copyright__ = """Copyright 2021, Matteo Lulli, Luca Biferale, 
Giacomo Falcucci, Mauro Sbragaglia and Xiaowen Shan, idea.deploy"""
__license__ = """Permission is hereby granted, free of charge, 
to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software 
without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, 
and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:
The above copyright notice and this permission notice shall be 
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE."""
__maintainer__ = "Matteo Lulli"
__email__ = "matteo.lulli@gmail.com"
__status__ = "Development"

In [2]:
# Development cell
%load_ext autoreload
%autoreload 2

# Metastable and Unstable Dynamics in multi-phase lattice Boltzmann

Authors: Matteo Lulli (1), Luca Biferale (2), Giacomo Falcucci (3), Mauro Sbragaglia (2), Dong Yang (1) and Xiaowen Shan (1)

(1) Department of Mechanics and Aerospace Engineering, Southern University of Science and Technology, Shenzhen, Guangdong 518055, China

(2) Department of Physics \& INFN, University of Rome \"Tor Vergata\", Via della Ricerca Scientifica 1, 00133, Rome, Italy.

(3) Department of Enterprise Engineering \"Mario Lucertini\", University of Rome \"Tor Vergata\", Via del Politecnico 1, 00133 Rome, Italy; John A. Paulson School of Engineering and Applied Physics, *Harvard University*,  33 Oxford Street, 02138 Cambridge, Massachusetts, USA.

**Abstract:**
We quantitatively characterize the metastability in a multi-phase lattice Boltzmann model. The structure factor of density fluctuations is theoretically obtained and numerically verified to a high precision, for all simulated wave-vectors and reduced temperatures. The static structure factor is found to consistently diverge as the temperature approaches the critical-point or the density approaches the spinodal line at a sub-critical temperature. Theoretically predicted critical exponents are observed in both cases. Finally, the phase separation in the unstable branch follows the same pattern, i.e. the generation of interfaces with different topology, as observed in molecular dynamics simulations. All results can be independently reproduced through the "idea.deploy" framework [https://github.com/lullimat/idea.deploy](https://github.com/lullimat/idea.deploy)

# Reproducibility

This document is intended for those interested readers who want to reproduce the results reported in the paper [https://arxiv.org/abs/2212.07848](https://arxiv.org/abs/2212.07848). In the present case the computational resources needed should be available in general. 

Next development steps will include a class to measure the time required by each cell and output it in a .json file which can be sent to [matteo.lulli@gmail.com](mailto:matteo.lulli@gmail.com) so that average execution times will be available and organized according to the hardware.

Each subsection can be executed independently and reproduce the results which will be stored locally in the directory 'reproduced-data', so that the data will be generated only once.

Plots can also be generated using the same scripts employed for the figures of the paper. Since there are some issues in executing these scripts in the Jupyter environment we include them as separated files which are called from the cells themselves. **In order to reproduce the plots a working 'latex' installation is necessary to be present on the system.**

This file will be kept updated for new local features and developments in the parent project [**idea.deploy**](https://github.com/lullimat/idea.deploy)

This file is supposed to be pulled from the repository [https://github.com/lullimat/arXiv-2212.07848](https://github.com/lullimat/arXiv-2212.07848), from within the "papers" directory the idea.deploy project.

# Paper Results
Using the next subsections/cells it is possible to reproduce all the results and plots reported in the paper.

## Simulations: Hardware Selection
In the following cells you are required to make a choice for the hardware to employ for the simulations. At the moment you need to stick to a given choice in order to visualize all the results. A more elastic management will be provided in future releases.

**These are the only cells in the notebook that have a "global" character, i.e. they are needed by the rest of the notebook for the simulations cell to run.**

By executing (shift-enter) the cell below, the hardware present in your system will be listed

In [3]:
# Display Hardware (click the arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import IdpyHardware

IdpyHardware()

OpenCL Found!

Listing GPUs:
OpenCL GPU[0]
Name :  Apple M1 Max
Memory :  22906503168
Double :  0
DrvVersion :  1.2 1.0


Listing CPUs:
OpenCL CPU[0]
Name :  Apple M1 Max
Memory :  34359738368
Double :  63
DrvVersion :  1.1


CTypes Found!

Listing CPUs:
Device: CPU Apple M1 Max Memory:34359738368




**To repdroduce the results you need hardware that support OpenCL with 64bits floating point variables (indicated by 'Double :  63') and/or CUDA.**

The variable "preferred_lang" can be set as
- OCL_T for OpenCL
- CUDA_T for CUDA

The variable "preferred_device" need to be set to the number matching the 64-bits floating point precision requirement discussed above.

The variable "preferred_kind" can be used only with OCL_T and can be set as
- "gpu" for selecting GPU devices
- "cpu" for selecting CPU devices

It also possible to run the simulations on 32-bits floating point variables, however the results will not match those presented in the paper.

In [4]:
# Language imports (click on left arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import CUDA_T, OCL_T, idpy_langs_sys

In [5]:
preferred_lang, preferred_device, preferred_kind = OCL_T, 0, "cpu"

In [ ]:
%%time

import sympy as sp

from idpy.LBM.SCFStencils import SCFStencils, BasisVectors


from idpy.Utils.SimpleTiming import SimpleTiming

_L = 96


_c_s2, _tau = 1/3, 1

n = sp.Symbol('n')
_psi_sym = sp.exp(-1/n)
_G = -3

_D2E4_P4F4 = SCFStencils(E = BasisVectors(x_max = 2), 
                         len_2s = [1, 2])
_D2E4_P4F4.FindWeights()

sc_eq_cache = ShanChanEquilibriumCache(stencil = _D2E4_P4F4, 
                                       psi_f = _psi_sym, 
                                       G = _G, 
                                       c2 = _c_s2, 
                                       n_eps = 1e-6)

_eq_params = sc_eq_cache.GetFromCache()

In [6]:
from idpy.LBM.SCFStencils import SCFStencils, BasisVectors
_D2E4_P4F4 = SCFStencils(E = BasisVectors(x_max = 2), 
                         len_2s = [1, 2])
_D2E4_P4F4.FindWeights()

from idpy.LBM.SCThermo import ShanChanEquilibriumCache, ShanChen

def GetDensityUnstablelDistance(a, psi, G, _stencil = _D2E4_P4F4):
    sc_eq_cache = ShanChanEquilibriumCache(stencil = _stencil, 
                                           psi_f = psi, 
                                           G = G, 
                                           c2 = 1/3, 
                                           n_eps = 1e-6)
    _eq_params = sc_eq_cache.GetFromCache()
    
    _sc = ShanChen(psi_f = psi, theta_val = 1/3, G_val = G)
    
    _density = (1 - a) * _sc.extrema[0][0] + a * _sc.extrema[1][0]    
    return _density

def UsePlotly(lbm, _eq_params, figures_dir, name):
    import plotly.graph_objects as go
    from plotly.figure_factory import create_quiver
    from functools import reduce
    from plotly.subplots import make_subplots

    _n_swap = lbm.GetDensityField()
    _L = lbm.sims_vars['dim_sizes'][0]

    _width = 800
    X, Y, Z = np.mgrid[0: _L, 0: _L, 0: _L]


    fig = go.Figure(data = [go.Volume(x = X.flatten(), y = Y.flatten(), z = Z.flatten(), 
                                     value = _n_swap.flatten(), 
                                     isomin = _eq_params['n_g'] * 0.999,
                                     isomax = _eq_params['n_l'] * 1.001,
                                     opacity = 0.22, 
                                     colorscale = [(0,"lightblue"), (1,"gray")], 
                                     opacityscale=[[_eq_params['n_g'] * 1., 0], 
                                                   [_eq_params['n_l'], 1]],
                                     showscale = False, surface_count=5)])

    fig.update_layout(scene_xaxis_showticklabels=False,
                      scene_yaxis_showticklabels=False,
                      scene_zaxis_showticklabels=False,
                      scene_xaxis_showline=False,
                      scene_xaxis_title = "", 
                      scene_yaxis_title = "", 
                      scene_zaxis_title = "")

    fig.update_layout(scene = dict(
                        xaxis = dict(
                             backgroundcolor="white",
                             gridcolor="white",
                             showbackground=False,
                             zerolinecolor="white",),
                        yaxis = dict(
                            backgroundcolor="white",
                            gridcolor="white",
                            showbackground=False,
                            zerolinecolor="white"),
                        zaxis = dict(
                            backgroundcolor="white",
                            gridcolor="white",
                            showbackground=False,
                            zerolinecolor="white",),),
                        width = _width, height = int(0.65 * _width), autosize = False, 
                        margin=dict(l=0, r=0, t=0, b=0)
                      )

    fig.write_image(str(figures_dir / (str(name) + '.png')))

    fig.show()

unstable_As = {'droplet': 0.15, 
               'cylinder droplet': 0.25, 
               'flat': 0.5, 
               'cylinder bubble': 0.93, 
               'bubble': 0.99}


unstable_seeds = {'droplet': 1, 
                  'cylinder droplet': 1, 
                  'flat': 1, 
                  'cylinder bubble': 113, 
                  'bubble': 1}

'''
Creating the directory for the figures
'''
from pathlib import Path

reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

'''
Decalring the simulations features
'''
from idpy.LBM.MultiPhase import ShanChenMultiPhase
from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
from idpy.IdpyStencils.IdpyStencils import IDStencils

_f_stencil = IDStencils['LBM']['SC_D3E4']
_xi_stencil = IDStencils['LBM']['XI_D3Q19']

_L, _G = 96, -3
_kBT = 1e-10
_c_s2, _tau = 1/3, 1

import sympy as sp

n = sp.Symbol('n')
psi_sym = sp.exp(-1/n)

psi_code = 'exp((NType)(-1./ln_0))'

'''
Single run timing
'''
from idpy.Utils.SimpleTiming import SimpleTiming
_st = SimpleTiming()

# Declaring simulation object _test_meta
unstable_sim = \
    ShanChenMultiPhase(dim_sizes = (_L, _L, _L), 
                       xi_stencil = _xi_stencil, 
                       f_stencil = _f_stencil,
                       psi_code = psi_code, 
                       psi_sym = psi_sym, 
                       e2_val = 1, 
                       SC_G = _G, tau = _tau,
                       lang = preferred_lang, device = preferred_device, 
                       cl_kind = preferred_kind, 
                       optimizer_flag = True, 
                       fluctuations = 'Gross2011', 
                       prng_distribution = 'gaussian', 
                       indep_gaussian = False, 
                       prng_kind = 'MMIX',
                       prng_init_from = 'numpy', 
                       init_seed = 1)

_a = unstable_As['droplet']
_n_start = GetDensityUnstablelDistance(_a, psi_sym, _G)

unstable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                             width = _L//2, direction = 0)

_st.Start()
unstable_sim.MainLoopGross2011SRT(
    time_steps = range(0, 2 ** 14 + 1, 2 ** 11), 
    convergence_functions = [CheckUConvergenceSCMP],
    profiling = False,
    kBT = _kBT, n0 = _n_start
)
_st.End()

_st.PrintElapsedTime()



unstable_sim.End()

OpenCL:  {'Name': 'Apple M1 Max', 'Device': '0', 'Memory': '34359738368', 'Kind': 'cpu', 'DrvVersion': '1.1', 'FP64': 63}
{'PopType': 'double', 'NType': 'double', 'UType': 'double', 'PsiType': 'double', 'ChiType': 'double', 'ThetaType': 'double', 'MuType': 'double', 'SCFType': 'double', 'NoiseType': 'double', 'ForceType': 'double', 'SType': 'int', 'WType': 'double', 'LengthType': 'double', 'FlagType': 'unsigned char'}
Extrema: 2.569999999999989
zero_ranges:  [(0.6599999999999974, 0.6699999999999973), (1.6099999999999934, 1.6199999999999934)]
Coexistence range (n, P):  [(0.3192298799060733, 0.103557611461254), (2.3948100527849987, 0.147548949618387)]

Step: 2048
u_conv:  0.00935581582050563 max_u:  0.042511328944885046
Conv! False True
Step: 4096
u_conv:  0.01361401277499865 max_u:  0.04058374703156508
Conv! False False
Step: 6144
u_conv:  0.008709156932932586 max_u:  0.007833311569510604
Conv! False False
Step: 8192
u_conv:  0.002002343066291407 max_u:  0.005303627179760726
Conv! False

NameError: name 'np' is not defined

In [ ]:
from idpy.IdpyCode import IdpyMemory
from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP

